In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import numpy as np
import plotly.graph_objs as go
import datetime
from dash.dependencies import Output, Input
import pandas as pd
import plotly.express as px
import time

app = dash.Dash()

np.random.seed(50)
x_rand = np.random.randint(1,61,60)
y_rand = np.random.randint(1,61,60)

df = pd.read_csv("transactions_10MIO.csv")

################################### handling date times for sliders

daterange = pd.date_range(start=min([x for x in df['lastout']]),end=max([x for x in df['lastout']]))#,freq='D')

def unixTimeMillis(dt):
    ''' Convert datetime to unix timestamp '''
    return int(time.mktime(dt.timetuple()))

def unixToDatetime(unix):
    ''' Convert unix timestamp to datetime. '''
    return pd.to_datetime(unix,unit='s')

def getMarks(start, end, Nth=100):
    ''' Returns the marks for labeling. 
        Every Nth value will be used.
    '''

    result = {}
    for i, date in enumerate(daterange):
        if(i%Nth == 1):
            # Append value to dict
            result[unixTimeMillis(date)] = str(date.strftime('%Y-%m-%d'))

    return result

#####################################

app.layout = html.Div([html.Div([html.H1("Wallet Explorer")], style={'textAlign': "center"}),
    html.Div([
        html.Div(
            dcc.Graph(
                id='scatter_chart',
                figure = {
                    # kann eigl weg
                    'data': go.Scatter(
                    ),
                    'layout' : go.Layout(
                        
                    )
                }
            )
        ),
        html.Label('date'),
        dcc.RangeSlider(
                id='slider_date',
                min = unixTimeMillis(daterange.min()),
                max = unixTimeMillis(daterange.max()),
                value = [unixTimeMillis(daterange.min()),
                         unixTimeMillis(daterange.max())],
                marks=getMarks(daterange.min(),
                            daterange.max()),
        ),
        html.Br(),
        html.Label('balance'),
        dcc.RangeSlider(
            id = 'slider_balance',
            min = min(df.weight),
            max = max(df.weight),
            value = [min(df.weight),max(df.weight)],
            step = 10,
            #marks = {i: i for i in range(max(df.weight))}
        ),
        html.Br(),
        html.Label('select owner type'),
        dcc.Dropdown(
            id = 'dropdown_type',
            options = [
                {'label' : 'Exchange', 'value' : 'exchange'},
                {'label' : 'Whale', 'value' : 'whale'},
                {'label' : 'Unknown', 'value' : 'unknown'},
                {'label' : 'All', 'value' : 'all'},
            ],
            value = 'exchange',
        ),
        html.Div(id='output-container-range-slider'),
    ],
        # not spanning the entire screen
    style={'width': '50%','display': 'inline-block'},
    )]
)

KeyError: 'lastout'

In [ ]:
@app.callback(
    dash.dependencies.Output('scatter_chart', 'figure'),
    [dash.dependencies.Input('slider_date', 'value'),dash.dependencies.Input('slider_balance', 'value'),
    dash.dependencies.Input('dropdown_type','value')],
    
)
    #[dash.dependencies.State('slider2', 'value')]) # value must be str, 
def update_date(value1, value2,value_type):
    global df
    df_filtered = df[df.category == value_type]
    df_filtered = df_filtered[(pd.to_datetime(df_filtered.lastout) < unixToDatetime(value1[1])) & (pd.to_datetime(df_filtered.lastout) > unixToDatetime(value1[0]))]
    df_filtered = df_filtered[(df_filtered.weight >= value2[0]) & (df_filtered.weight <= value2[1])]
    data = go.Scatter(
                    x = df_filtered.lastout,
                    y = df_filtered.firstin,
                    mode = 'markers',
                    marker= {'size': [x for x in df_filtered.weight[(df_filtered.weight >= value2[0]) & (df_filtered.weight <= value2[1])]]}
    )
    layout = go.Layout(
                        title = "Transactions",
                        xaxis = {'title' : 'First In'},
                        yaxis = {'title' : 'Last Out'},)
    fig=go.Figure(data=data, layout=layout)
    return fig#print('From {} to {}'.format(unixToDatetime(value1[0]),unixToDatetime(value1[1])),value1)
    
    #return 'You have selected "{}"'.format(value)
    

if __name__ == "__main__":
    app.run_server(port=4050)

In [ ]:
df.lastout[pd.to_datetime(df.lastout) > unixToDatetime(1516503843)]

In [54]:
value2=[10,30]
x = df.weight[(df.weight >= value2[0]) & (df.weight <= value2[1])]
print(a.bool(x))


AttributeError: 'list' object has no attribute 'bool'

In [48]:
a = list(map(lambda x: (x >= value2[0]) & (x <= value2[1]),df.weight))
a

[True, True, False, True, True]

In [118]:
df = df[df.category == 'whale']
df

,number,lastout,firstin,weight,category
1,2,2019-10-16,2019-11-01,10,whale
2,3,2019-09-10,2019-11-03,40,whale


In [143]:
df.head()


,hash,block_timestamp,sender,receiver,btc,date,sender_name,sender_category,receiver_name,receiver_category,CapMrktCurUSD,PriceUSD,dollar,percent_marketcap,sender_name2,receiver_name2
0,cffb265addcda4aaaa6e9b43e9c7ee7fe6ec962d7f0058...,2018-07-20 01:47:36+00:00,36R8aHHGCd7GE5fJEHRuvoUn2ADkfsVLQD,3ApkoC44wdAytaWuFZ4yevLJZPvxYQLPYX,1478.225872,2018-7-20,NaN,NaN,NaN,NaN,1.257407e+11,7327.256683,1.083134e+07,0.008614,unknown,unknown
1,cffb265addcda4aaaa6e9b43e9c7ee7fe6ec962d7f0058...,2018-07-20 01:47:36+00:00,3DTQcTKDmFBGvp3JT8Ak2RZ1sgumNixJHB,3ApkoC44wdAytaWuFZ4yevLJZPvxYQLPYX,1478.225872,2018-7-20,NaN,NaN,NaN,NaN,1.257407e+11,7327.256683,1.083134e+07,0.008614,unknown,unknown
2,cffb265addcda4aaaa6e9b43e9c7ee7fe6ec962d7f0058...,2018-07-20 01:47:36+00:00,33Pr9krFzEFs3zcHstLwJtJuX2aAYE7vW5,3ApkoC44wdAytaWuFZ4yevLJZPvxYQLPYX,1478.225872,2018-7-20,NaN,NaN,NaN,NaN,1.257407e+11,7327.256683,1.083134e+07,0.008614,unknown,unknown
3,cffb265addcda4aaaa6e9b43e9c7ee7fe6ec962d7f0058...,2018-07-20 01:47:36+00:00,3Cr5rvJX7uWL8uNiwXZX6szSx2UAHNwfKv,3ApkoC44wdAytaWuFZ4yevLJZPvxYQLPYX,1478.225872,2018-7-20,NaN,NaN,NaN,NaN,1.257407e+11,7327.256683,1.083134e+07,0.008614,unknown,unknown
4,cffb265addcda4aaaa6e9b43e9c7ee7fe6ec962d7f0058...,2018-07-20 01:47:36+00:00,3Avx3jF2aLVkgSeMRv7HEmrjVmupZW5wkm,3ApkoC44wdAytaWuFZ4yevLJZPvxYQLPYX,1478.225872,2018-7-20,NaN,NaN,NaN,NaN,1.257407e+11,7327.256683,1.083134e+07,0.008614,unknown,unknown
